# LangChain と PaLM API で PDF ドキュメントの要約を行う例

LangChain のパッケージ、および、PDF の扱いに必要なパッケージをインストールします。

In [ ]:
pip install langchain==0.0.294 pypdf==3.16.1 cryptography==41.0.4

NISC が一般公開している「[インターネットの安全・安心ハンドブック](https://security-portal.nisc.go.jp/guidance/handbook.html)」のプロローグ部分をダウンロードします。





In [1]:
!wget -q -O handbook.pdf \
  https://security-portal.nisc.go.jp/guidance/pdf/handbook/handbook-prologue.pdf

ダウンロードした PDF を読み込んで、全体を1つのテキストにまとめます。

In [2]:
from langchain.document_loaders import PyPDFLoader

pages = PyPDFLoader('handbook.pdf').load()
document = ''
for page in pages:
    document += page.page_content

PaLM API を利用するクライアントオブジェクトを取得します。

In [3]:
from langchain.llms import VertexAI
llm = VertexAI(model_name='text-bison@001',
               temperature=0.1, max_output_tokens=1024)

RecursiveCharacterTextSplitter, load_qa_chain, AnalyzeDocumentChain モジュールの使用例です。

`input_document` に与えたテキストに基づいて、`question` の質問に回答します。

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='map_reduce')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

qa_document_chain.run(
    input_document='今は６月で梅雨の時期です。', question='最近の天候は？')

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (1468 > 1024). Running this sequence through the model will result in indexing errors


'梅雨の時期です。'

`input_document` に与えたテキストの内容をまとめるように指示して結果を得る関数を用意します。

In [5]:
def get_description(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='map_reduce')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '何についての文書ですか？日本語で5文程度にまとめて教えてください。'
    description = qa_document_chain.run(
        input_document=document, question=prompt)
    return description

先に用意した PDF ファイルを適用します。

In [11]:
print(get_description(document))

この文書は、インターネットの安全・安心に関する基本的なリスクやトラブルについて解説したハンドブックです。サイバー攻撃の種類やその手口、スマホやパソコン、IoT機器を安全に利用するための設定などについて解説しています。


任意の質問に答える関数を用意します。

In [7]:
def get_answer(document, question):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='refine')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '{} 日本語で5文程度にまとめて教えてください。'.format(question)
    answer = qa_document_chain.run(
        input_document=document, question=prompt)
    return answer

先に用意した PDF ファイルに基づいて、質問の回答を取得します。

In [14]:
question = 'サイバーセキュリティ対策のポイントを箇条書きにまとめてください。'
print(get_answer(document, question))

サイバーセキュリティ対策のポイントは以下の通りです。
1. 最新の攻撃情報に対抗するため、OSやソフトウェアメーカーが提供している修正用アップデートを常に適用しましょう。
2. パスワードは長く複雑にして、機器やサービス間で使い回さないことを徹底して安全性を高めましょう。
3. サービスへのログインを安全に行うために、認証用アプリや生体認証を使った多要素認証を利用しましょう。
4. 偽メールや偽サイトに騙されないように用心しましょう。
5. メールの添付ファイルや本文中のリンクに注意しましょう。
6. スマホやPCの画面ロックを活用しましょう。
7. 大切な情報は失う前にバックアップ（複製）しましょう。
8. 外出先では紛失・盗難・覗き見に注意しましょう。
9. 困ったときは1人で悩まず、まず相談しましょう。


PDF ファイルを使用せず、PaLM API を直接使用した場合の回答を確認します。

In [9]:
from vertexai.preview.language_models import TextGenerationModel
generation_model = TextGenerationModel.from_pretrained('text-bison@001')

def get_response(prompt, temperature=0.1):
    response = generation_model.predict(
         prompt, temperature=temperature, max_output_tokens=1024)
    return response.text

In [10]:
prompt = '''\
サイバーセキュリティ対策のポイントを箇条書きにまとめてください。
日本語で5文程度にまとめて教えてください。
'''
print(get_response(prompt))

サイバーセキュリティ対策のポイントは以下のとおりです。

* 強固なパスワードの使用
* 二要素認証の設定
* ソフトウェアのアップデートの適用
* 不審なメールやリンクの開かない
* データのバックアップの取得

これらの対策を実施することで、サイバー攻撃のリスクを大幅に減らすことができます。
